In [1]:
# note celerite is linear with N, so no benefit to breaking into chunks
# if I wanted to fit two datasets at the same time, I would need to 
# create a new jitter term that has different variances for the the
# different data sets. Harder if assume different amplitudes for two
# datasets.

%matplotlib inline

In [2]:
import emcee
import autograd.numpy as np
import matplotlib.pyplot as plt
from astropy.stats import LombScargle, median_absolute_deviation
from scipy.optimize import minimize
import glob
from tqdm import tqdm
import corner

import celerite
from celerite import terms

from gp import get_rotation_gp
from astropy.io import fits
from statsmodels import robust
import k2plr

do_kep = False
do_kep2 = False
do_kep3 = True
do_test = False

In [3]:

if do_kep:
    f = fits.open('../data/kplr009726699-2010203174610_slc.fits') 
    hdu_data = f[1].data

    t = hdu_data["time"]
    y = hdu_data["sap_flux"]/np.nanmedian(hdu_data["sap_flux"])-1
    yerr = hdu_data["sap_flux_err"]/np.nanmedian(hdu_data["sap_flux"])

    ninc = 5000
    rand = 0 #np.random.randint(0,len(t)-ninc)
    t = t[rand:rand+ninc]
    y = y[rand:rand+ninc]
    yerr = yerr[rand:rand+ninc]

    name = f[0].header['OBJECT'].replace(' ','')

elif do_kep2:
    f = fits.open('../data/kplr009726699-2009350155506_llc.fits') 
    hdu_data = f[1].data

    t = hdu_data["time"]
    y = hdu_data["sap_flux"]/np.nanmedian(hdu_data["sap_flux"])-1
    yerr = hdu_data["sap_flux_err"]/np.nanmedian(hdu_data["sap_flux"])

    #ninc = 5000
    #rand = 0 #np.random.randint(0,len(t)-ninc)
    #t = t[rand:rand+ninc]
    #y = y[rand:rand+ninc]
    #yerr = yerr[rand:rand+ninc]

    mask = hdu_data["sap_quality"] == 0
    t = t[mask]
    y = y[mask]
    yerr = yerr[mask]
    
    name = f[0].header['OBJECT'].replace(' ','')

elif do_kep3:
    k = 9726699
    kclient = k2plr.API()
    if k>100000000: 
        star = kclient.k2_star(k) ## K2
    else:
        star = kclient.star(k) # Kepler

    lcs = star.get_light_curves(short_cadence=False)

    quarters = np.zeros_like(lcs, dtype=int)
    for i, lc in enumerate(lcs):
        hdu_list = lc.open()
        quarters[i] = hdu_list[0].header['QUARTER']
        hdu_list.close()

    qq, = np.where(quarters == 9)
    
    lc = lcs[qq[0]]
    with lc.open() as f:
        hdu_data = f[1].data
        time = hdu_data["time"]
        flux = hdu_data["sap_flux"]/np.nanmedian(hdu_data["sap_flux"])-1.
        ferr = hdu_data["sap_flux_err"]/np.nanmedian(hdu_data["sap_flux"])
        mask = hdu_data["sap_quality"] == 0
        name = f[0].header['OBJECT'].replace(' ','')
        
        t = t[mask]
        y = y[mask]
        yerr = yerr[mask]
   
    
    
elif do_test:
    
    t, y = np.genfromtxt('/Volumes/Mimas/full_dataset/final/lightcurve_0289.txt',
                        unpack=True)
    y = y/np.median(y) - 1.
    err_est = robust.mad(np.diff(y)) 
    ## should multiple by 1.48, and then divide by sqrt(2) so pretty much good as is
    yerr = np.ones_like(y)*err_est

    ninc = 5000
    rand = 0 #np.random.randint(0,len(t)-ninc)
    t = t[rand:rand+ninc]
    y = y[rand:rand+ninc]
    yerr = yerr[rand:rand+ninc]
    
    name = '0289'
    
else:
    
    t, y, yerr = np.genfromtxt('example_star.csv', delimiter=',', unpack=True)
    name = 'example'


In [4]:
# Do some aggressive sigma clipping
m = np.ones_like(y, dtype=bool)
while True:
    mu = np.nanmean(y[m])
    sig = np.nanstd(y[m])
    m0 = y - mu < 3 * sig
    if np.all(m0 == m):
        break
    m = m0

m = m*np.isfinite(y)*np.isfinite(yerr)*np.isfinite(t)
t_orig, y_orig, yerr_orig = np.copy(t), np.copy(y), np.copy(yerr)
f = lambda x: np.ascontiguousarray(x, dtype=np.float64)
t, y, yerr = map(f, [t[m], y[m], yerr[m]])

NameError: name 'y' is not defined

In [ ]:
print (len(t_orig), len(y_orig))
plt.plot(t_orig, y_orig, '.')
plt.plot(t, y, '.')
#plt.xlim(320, 330)

In [ ]:
# First guess at the period
fmin = max([2./(t[-1]-t[0]),0.02] )
freq = np.linspace(fmin, 10.0, 5000)
model = LombScargle(t, y)
power = model.power(freq, method="fast", normalization="psd")
power /= len(t)

period = 1.0 / freq[np.argmax(power)]
print("LS period", period)

plt.plot(1.0 / freq, power, "k")
plt.axvline(period)
plt.xscale("log")
plt.yscale("log")

plt.figure()
plt.plot(t % period, y, ".k")


In [ ]:
#t, y, yerr = t_orig, y_orig, yerr_orig
min_period = period * 0.8
max_period = period / 0.8

gp = get_rotation_gp(t, y, yerr,
                     period, min_period, max_period)
gp.compute(t, yerr)
print len(t), np.sum(y), np.sum(yerr)
print period, min_period, max_period
print(gp.log_likelihood(y))
gp.get_parameter_dict()

In [ ]:
def neg_log_like(params, y, gp, m):
    gp.set_parameter_vector(params)
    return -gp.log_likelihood(y[m])

def grad_neg_log_like(params, y, gp, m):
    gp.set_parameter_vector(params)
    return -gp.grad_log_likelihood(y[m])[1]


# Do another round of sigma clipping using the GP model
# freeze the rotation period so it doesn't fit flares
gp.freeze_parameter("kernel:terms[2]:log_P")
#gp.freeze_parameter("kernel:terms[0]:log_S0")
#gp.freeze_parameter("kernel:terms[0]:log_omega0")
#gp.freeze_parameter("kernel:terms[1]:log_sigma")
#gp.freeze_parameter("kernel:terms[2]:log_a")
#gp.freeze_parameter("kernel:terms[2]:log_Q2")
#gp.freeze_parameter("kernel:terms[2]:log_Q1")

initial_params = gp.get_parameter_vector()
bounds = gp.get_parameter_bounds()

# t, y, yerr = t_orig, y_orig, yerr_orig
m = np.ones(len(t), dtype=bool)
for i in range(2):
    plt.figure()
    plt.plot(t[m], y[m], ".k")
    ylim = plt.gca().get_ylim()

    gp.compute(t[m], yerr[m])   ## to figure out the shape of 
                                ## the array and time stamps
                                ## factorizes the covariance matrix
    soln = minimize(neg_log_like, initial_params, jac=grad_neg_log_like,
                    method="L-BFGS-B", bounds=bounds, args=(y, gp, m))
    gp.set_parameter_vector(soln.x) ## this also re-computes
    initial_params = soln.x

    mu, var = gp.predict(y[m], t, return_var=True)
    plt.plot(t, mu, zorder=0)
    plt.ylim(ylim)
    #plt.ylim(-1,2)
    
    resid = y - mu
    #sig = np.sqrt(np.median(resid**2))
    sig = np.sqrt(var + yerr**2)
    m0 = resid < 1.3 * sig
    print(m0.sum(), m.sum())
    if np.all(m0 == m) or (np.abs(m0.sum()- m.sum()) < 3):
        break
    m = m0

gp.thaw_parameter("kernel:terms[2]:log_P")

fit_t, fit_y, fit_yerr = t[m], y[m], yerr[m]
gp.compute(fit_t, fit_yerr)
gp.get_parameter_dict()

In [ ]:
omega = np.exp(np.linspace(np.log(0.1), np.log(20), 5000))
psd = gp.kernel.get_psd(omega)

plt.plot(omega, psd)
for k in gp.kernel.terms:
    plt.plot(omega, k.get_psd(omega), "--")

plt.yscale("log")
plt.xscale("log")
plt.xlim(omega[0], omega[-1])
plt.xlabel("$\omega$")
plt.ylabel("$S(\omega)$");

In [ ]:
np.random.seed(82)
def log_prior(params, logperiod, pp=False):
    lp = gp.log_prior()
    if pp: print lp
    p = gp.get_parameter_dict()['kernel:terms[2]:log_P']
    period = np.exp(logperiod)
    sigma = 0.2
    
    logperiod_half = logperiod + np.log(0.5)
    logperiod_twice = logperiod + np.log(2.)
    gaussian_prior = (-1./2.)*((p - logperiod)/(sigma))**2.
    gaussian_prior_half = (-1./2.)*((p - logperiod_half)/(sigma))**2.
    gaussian_prior_twice = (-1./2.)*((p - logperiod_twice)/(sigma))**2.

    lp = lp + 0.5*gaussian_prior + 0.25*gaussian_prior_half + 0.25*gaussian_prior_twice
    
    if (np.abs(p-logperiod)>0.4) & (np.abs(p-logperiod_half)>0.4) & (np.abs(p-logperiod_twice)>0.4):
        return -np.inf
    
    if pp: print lp
    return lp

def log_prob(params, logperiod):
    gp.set_parameter_vector(params)
    lp = log_prior(params, logperiod)
    if not np.isfinite(lp):
        return -np.inf
#     make the noise budget be dominated by the peroidic signal
#     if (gp.get_)
    return lp + gp.log_likelihood(fit_y)

logperiod = gp.get_parameter_dict()['kernel:terms[2]:log_P']
initial_params = gp.get_parameter_vector()
print "starting", gp.log_likelihood(fit_y)
print log_prob(initial_params, logperiod)
print log_prior(initial_params, logperiod, pp=True)

ndim = len(initial_params)
nwalkers = 64
print(gp.get_parameter_dict()    )

# set up initial positions
pos = initial_params + 1e-2 * np.random.randn(nwalkers, ndim)

# but for period start some at the harmonics
tmp = [name == 'kernel:terms[2]:log_P' for name in gp.get_parameter_names()]
perloc = np.where(tmp)[0][0]
for i in range(nwalkers):
    myrand = np.random.uniform()
    if myrand < 0.25:
        pos[i][perloc] = logperiod + np.log(0.5) + 1e-2 * np.random.randn()
    elif myrand < 0.5:
        pos[i][perloc] = logperiod + np.log(2) + 1e-2 * np.random.randn()
            

# and make sure none of them are NaNs
lp = np.array( [log_prob(pos_i, logperiod) for pos_i in pos] )
m = ~np.isfinite(lp)
while np.any(m):
    print "val", pos[i][perloc]
    pos[m] = initial_params + 1e-3 * np.random.randn(m.sum(), ndim)
    #lp[m] = np.array(list(map(log_prob, pos[m])))
    lp[m] = np.array( [log_prob(pos_i, logperiod) for pos_i in pos[m]] )
    m = ~np.isfinite(lp)

args=[logperiod]
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_prob, args=args)
pos, _, _ = sampler.run_mcmc(pos, 500)
print "burn", gp.log_likelihood(fit_y)


In [ ]:
#sampler.reset()
#sampler.run_mcmc(pos, 2000);
#print "chain", gp.log_likelihood(fit_y)


In [ ]:
m = np.isfinite(y_orig)*np.isfinite(yerr_orig)*np.isfinite(t_orig)
mle = sampler.flatchain[np.argmax(sampler.flatlnprobability)]
gp.set_parameter_vector(mle)
mu, var = gp.predict(fit_y, t_orig, return_var=True)
std = np.sqrt(yerr_orig[m]**2 + var)


In [ ]:
pdist = np.exp(sampler.flatchain[:, 7])
pbest = np.median(pdist)
               
fig = plt.figure(figsize=[11,6])
color = "#ff7f0e"

ax1 = plt.subplot2grid((2, 3), (1, 0), colspan=2, )
ax2 = plt.subplot2grid((2, 3), (1, 2), colspan=1, )

ax1.fill_between(t_orig[m], mu+std*3, mu-std*3, alpha=0.7, color=color, zorder=1)
ax1.plot(t_orig, y_orig, '.-', zorder=0)
ax1.set_ylim(-0.02,0.03)
xl = [t_orig[0], t_orig[-1]]
#if (xl[1]-xl[0]) > 8*pbest:
#    xl = [t_orig[0]+16*pbest, t_orig[0]+16*pbest]
ax1.set_xlim(xl)
ax1.set_xlim(300,320)
ax1.set_xlabel('Time (days)', fontsize=14)
ax1.set_ylabel('Relative Brighness', fontsize=14)

ax2.hist(pdist, 50, histtype="step")
ax2.set_xlabel('Rotation Period (days)', fontsize=14)
ax2.set_ylabel('Posterior Probability', fontsize=14)

fig.tight_layout()

In [ ]:
varnames = gp.get_parameter_dict().keys()
samples = sampler.chain[:, :, :].reshape((-1, ndim)) 
best = map(lambda v: [v[1], v[2]-v[1], v[1]-v[0]], \
                   zip(*np.percentile(samples, [16, 50, 84], axis=0))) ## arranged: [50th, uppe
mydict = {}
labels = [None]*ndim
for i in range(len(varnames)):
    vv = varnames[i][16:]
    if vv == 'mix_par':
        vv = 'mix'
    else:
        vv = vv.replace('log_','log(')+')'
    mydict[vv] = best[i]
    labels[i] = vv
mydict

In [ ]:
#messing around to try and visualize what's going on
# but this doesn't work because each term is trying to explain
# all of what's going on since it depends on the input y values
from mixterm import MixtureOfSHOsTerm
kernel = gp.get_parameter_dict(include_frozen=True)

period_kernel = MixtureOfSHOsTerm(
        log_a=kernel['kernel:terms[2]:log_a'], ## amplitude of the main peak
        log_Q1=kernel['kernel:terms[2]:log_Q1'], ## decay timescale of the main peak (width of the spike in the FT)
        mix_par= kernel['kernel:terms[2]:mix_par'], ## height of second peak relative to first peak
        log_Q2=kernel['kernel:terms[2]:log_Q2'], ## decay timescale of the second peak
        log_P=kernel['kernel:terms[2]:log_P'], ## period (second peak is constrained to twice this)
        bounds=dict(
            log_a=(-20.0, 10.0),
            log_Q1=(0., 10.0),
            mix_par=(-5.0, 10.0),
            log_Q2=(0., 10.0),
            log_P=(None, None), # np.log(min_period), np.log(max_period)),
        )
    )
period_gp = celerite.GP(kernel=period_kernel, mean=np.nanmean(fit_y))
period_gp.compute(fit_t, fit_yerr)
mu_period, _ = period_gp.predict(fit_y, t_orig[m], return_var=False)


basic_kernel = terms.SHOTerm(
    log_S0=kernel['kernel:terms[0]:log_S0'],
    log_Q=kernel['kernel:terms[0]:log_Q'],
    log_omega0=kernel['kernel:terms[0]:log_omega0'],
    bounds=dict(
        log_S0=(-20.0, 10.0),
        log_omega0=(np.log(2*np.pi/(period*50.)), np.log(2*np.pi/(period*10))),
    ),
)
basic_gp = celerite.GP(kernel=basic_kernel, mean=np.nanmean(fit_y))
basic_gp.compute(fit_t, fit_yerr)
mu_basic, _ = basic_gp.predict(fit_y, t_orig[m], return_var=False)

jitter_kernel = terms.JitterTerm(log_sigma=kernel['kernel:terms[1]:log_sigma'],
                               bounds=[(-20,20)])
jitter_gp = celerite.GP(kernel=jitter_kernel, mean=np.nanmean(fit_y))
jitter_gp.compute(fit_t, fit_yerr)
mu_jitter, _ = jitter_gp.predict(fit_y, t_orig[m], return_var=False)


In [ ]:
fig = plt.figure(figsize=[11,6])
color = "#ff7f0e"
ax1 = plt.subplot()
ax1.plot(t_orig[m], mu_period, alpha=0.7, lw=3, color='C0', zorder=1, label='SHO Mixture')
ax1.plot(t_orig[m], mu_basic, alpha=0.7, lw=3, color='C4', zorder=1, label='SHO')
ax1.plot(t_orig[m], mu_jitter, alpha=0.7, lw=3, color='C3', zorder=1, label='Jitter')
ax1.plot(t_orig[m], mu, ":", alpha=0.7, lw=3, color=color, zorder=2)
#ax1.plot(t_orig, y_orig, '.-', zorder=0)
ax1.set_xlim(0,10)
plt.legend()

In [ ]:
# Make the base corner plot
figure = corner.corner(samples)

# Extract the axes
axes = np.array(figure.axes).reshape((ndim, ndim))

# Loop over the diagonal
for i in range(ndim):
    ax = axes[i, i]
    print ax
    low, high = np.percentile(samples[:,i], [2,98])
    print low, high
    ax.set_xlim(low, high)


In [ ]:
toplot = np.array(np.linspace(0,len(sampler.chain[0,:,0])-1,1000), dtype=int)
plt.plot(sampler.chain[:, toplot, 7].T, color="k", alpha=0.4);

In [ ]:
np.log(0.5/2)